RAG model using BERT and LangChain as the retriever and BLOOM as the generator.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [ ]:
!pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53895 sha256=9bd1ec9c0a4f72de63d7592d59217d751bd817f6f6dd43f0de1ff3d0dd053c93
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [ ]:
!pip install python-docx
from docx import Document


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.7 MB/s eta 0:00:00


In [2]:
from docx import Document

# Load the document
doc_path = "/content/sample_data/Hair Document.docx"  # Update the path if necessary


In [3]:
doc = Document(doc_path)

In [4]:
# Extract text from the document
doc_text = ""
for paragraph in doc.paragraphs:
    doc_text += paragraph.text + "\n"

In [5]:
# Display the first few characters of the document text
print(doc_text[:200])

Hair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual. Significant progress is being made on discovering an effectiv


In [6]:
import re

# Perform basic preprocessing: remove special characters and lower the text
doc_text_cleaned = re.sub(r'[^a-zA-Z\s]', '', doc_text).lower()

# Tokenize the text into sentences or passages
sentences = doc_text_cleaned.split('\n')

# Display the first few sentences
for sentence in sentences[:5]:
    print(sentence)


hair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual significant progress is being made on discovering an effective and safe drug for hair growth angiogenesis androgen antagonism vasodilation potassium channel opening and alpha reductase inhibition are the major nonsurgical therapeutic strategies of hair growth promotion in spite of a flood of drugs claiming to be useful as hair growth promoters more rational strategies which can target the problem areas or stages of the hair growth cycle effectively are still awaited this article highlights the developments in hair rejuvenation strategies and reviews the potential of herbal drugs as safer and effective alternatives

hair has been a sign of beauty and a contribution to an individuals personality since time immemorial alopecia baldness a dermatological disorder is a common problem of cosmetology as well as primary health practice it is common throughout

In [7]:
# Load the question-answer pairs document
qna_pairs_path = "/content/sample_data/Hair qna pairs.docx"  # Update the path if necessary
qna_doc = Document(qna_pairs_path)

# Extract text from the question-answer pairs document
qna_doc_text = ""
for paragraph in qna_doc.paragraphs:
    qna_doc_text += paragraph.text + "\n"

# Perform basic preprocessing: remove special characters and lower the text
qna_doc_text_cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', qna_doc_text).lower()

# Tokenize the text into question-answer pairs
qna_pairs = [pair.split('\n') for pair in qna_doc_text_cleaned.split('\n\n')]

# Display the first few question-answer pairs
for qna_pair in qna_pairs[:3]:
    print(qna_pair)


['hair question and answer pairs']
['question what are the major nonsurgical therapeutic strategies for promoting hair growth', 'answer the major nonsurgical therapeutic strategies for promoting hair growth include angiogenesis androgen antagonism vasodilation potassium channel opening and 5alpha reductase inhibition', 'question what is the significance of herbal drugs in hair growth promotion', 'answer herbal drugs have been considered safer and effective alternatives for promoting hair growth since ancient times and are widely used in ayurveda chinese medicine and unani systems', 'question what are the two us fdaapproved synthetic drugs for treating alopecia', 'answer minoxidil and finasteride are the two us fdaapproved synthetic drugs for treating alopecia although their side effects have limited their usage', 'question what are the phases of the hair growth cycle', 'answer the phases of the hair growth cycle are anagen growth catagen regression telogen resting and exogen shedding',

In [8]:
from transformers import BertForQuestionAnswering, GPT2LMHeadModel

# Load pre-trained BERT model for question-answering
bert_model_name = "bert-base-uncased"  # You can also use other variants if needed
bert_retriever = BertForQuestionAnswering.from_pretrained(bert_model_name)

# Load pre-trained BLOOM (GPT) model for text generation
bloom_model_name = "gpt2-medium"  # You can adjust the model size as needed
bloom_generator = GPT2LMHeadModel.from_pretrained(bloom_model_name)

# Display model configurations
print("BERT Model Configuration:")
print(bert_retriever.config)

print("\nBLOOM Model Configuration:")
print(bloom_generator.config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Model Configuration:
BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}


BLOOM Model Configuration:
GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type"

In [9]:
# Extract questions and answers from the question-answer pairs
questions = []
answers = []

for qna_pair in qna_pairs:
    if len(qna_pair) == 2:  # Ensure it's a valid question-answer pair
        question, answer = qna_pair
        questions.append(question.strip())
        answers.append(answer.strip())

# Format the data into a format suitable for training BERT
# You may need to further preprocess the data based on BERT's requirements
# For example, you may need to add special tokens ([CLS], [SEP]) and perform tokenization

# Display the first few questions and answers
for i in range(3):
    print(f"Question {i+1}: {questions[i]}")
    print(f"Answer {i+1}: {answers[i]}")
    print()


Question 1: question what are the main factors affecting hair growth
Answer 1: answer hair growth is influenced by various factors such as ethnicity gender scalp area natural hair color and hair curliness

Question 2: question what were the findings regarding hair density across different scalp areas
Answer 2: answer the study found that hair density varied across different scalp areas with the vertex area having the highest density followed by the nape and temple

Question 3: question did natural hair color have any impact on hair growth parameters
Answer 3: answer while natural hair color did not significantly affect total hair density and growth rate there was a tendency for telogen percentage t to decrease with lighter hair tones



In [10]:
# Load the text from the "Hair Document" file
hair_doc_path = "/content/sample_data/Hair Document.docx"  # Update the path if necessary
hair_doc = Document(hair_doc_path)

# Extract text from the "Hair Document"
hair_doc_text = ""
for paragraph in hair_doc.paragraphs:
    hair_doc_text += paragraph.text + "\n"

# Tokenize the text into sentences or passages suitable for retrieval
hair_doc_sentences = hair_doc_text.split("\n")  # Split by newline character for sentences

# Display the first few sentences
for i in range(3):
    print(f"Sentence {i+1}: {hair_doc_sentences[i]}")
    print()


Sentence 1: Hair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual. Significant progress is being made on discovering an effective and safe drug for hair growth. Angiogenesis, androgen antagonism, vasodilation, potassium channel opening and 5-alpha reductase inhibition are the major non-surgical therapeutic strategies of hair growth promotion. In spite of a flood of drugs claiming to be useful as hair growth promoters, more rational strategies, which can target the problem areas or stages of the hair growth cycle effectively, are still awaited. This article highlights the developments in hair rejuvenation strategies and reviews the potential of herbal drugs as safer and effective alternatives.

Sentence 2: 

Sentence 3: Hair has been a sign of beauty and a contribution to an individual’s personality since time immemorial. Alopecia (baldness), a dermatological disorder, is a common problem of cosmetology 

In [11]:
from transformers import TrainingArguments

In [12]:
# Define training parameters for BERT
training_args = TrainingArguments(
    output_dir='./output',  # Add output directory
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
)

In [14]:
from transformers import AdamW

In [16]:
from transformers import AdamW, BertTokenizer, BertModel

# Import other necessary components

# Define your BERT model
model = BertModel.from_pretrained('bert-base-uncased')

# Define your tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define your optimizer
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

# Continue with your project


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
